In [4]:
import pandas as pd
import datetime
import os

## Select stations
Stations considered for clustering are those that appear in all files. 

In [80]:
# LOAD DATASETS
stations_path = "datasets/estacions/"
info_path = "datasets/informacio/"
stations_data = {}
info_data = {}

for stations, info in zip(sorted(os.listdir(stations_path)), sorted(os.listdir(info_path))):
    try: 
        assert(stations[:7] == info[:7])
        stations_data[stations[:7]] = pd.read_csv(stations_path+stations)
        info_data[info[:7]] = pd.read_csv(info_path+info)
    except:
        print(stations, info)
try:
    stations_data.pop(".gitatt")
    info_data.pop(".gitatt")
except: 
    pass

/usr/local/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (7,11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [73]:
stations = []
for doc in stations_data:
    if doc in ['2019_04','2019_05','2019_06', '2019_07','2019_08','2019_09', '2019_10','2019_11','2019_12','2020_01','2020_02','2020_03']:
        stations.append(set(stations_data[doc]["station_id"].unique()))

In [74]:
common_stations = stations[1].intersection(*stations) 

## One month average

In [79]:
stations_data['2019_]

{'.gitatt': Empty DataFrame
 Columns: [*.csv filter=lfs diff=lfs merge=lfs -text]
 Index: [], '2019_04':          station_id  num_bikes_available  \
 0                 1                   25   
 1                 2                   22   
 2                 3                   22   
 3                 4                   15   
 4                 5                   38   
 ...             ...                  ...   
 3018519         428                    2   
 3018520         492                    9   
 3018521         494                    0   
 3018522         495                   12   
 3018523         496                   15   
 
          num_bikes_available_types.mechanical  \
 0                                          25   
 1                                          22   
 2                                          22   
 3                                          15   
 4                                          38   
 ...                                       ...   
 301

In [78]:
monthly_means = {}
for month in stations_data:
    # Get dataframe and drop columns not needed - "static" information about the stations
    data = stations_data[month]
    print()
    data = data.drop(columns = ["is_installed", "is_renting", "is_returning", "last_reported", "is_charging_station", "status", "ttl"], inplace=False)
    # Get day and hour of each record 
    data["day"] = data["last_updated"].apply(lambda x: datetime.datetime.fromtimestamp(x).strftime("%d"))
    data["hour"] = data["last_updated"].apply(lambda x: datetime.datetime.fromtimestamp(x).strftime("%H:%M:%S"))   
    data = data.drop(columns = ["last_updated"])
    
    # Compute the mean for each station
    monthly_mean = pd.DataFrame(columns = ["station_id", "num_bikes_available", 
                                       "num_bikes_available_types.mechanical", 
                                       "num_bikes_available_types.ebike", "num_docks_available", "hour"])
    
    for station_id in common_stations:
        station_df = data[data["station_id"] == station_id]
        if len(station_df) >= 288 * 20: # minimum number of rows per station_id 
            for i in range(0, 24, 3):
                    hours = ["{:02d}".format(i), "{:02d}".format(i+1), "{:02d}".format(i+2)]
                    temp_df = station_df[station_df["hour"].str[:2].isin(hours)]
                    mean = temp_df.mean().drop(labels=["day"])
                    mean["hour"] = hours[0] + ":00 - " + hours[-1]+":55"
                    mean["station_id"] = int(mean["station_id"])
                    monthly_mean = monthly_mean.append(mean, ignore_index = True)
                    
        else: 
            print(station_id)
            not_used_stations.append(station_id)

KeyError: "['is_installed' 'is_renting' 'is_returning' 'last_reported'\n 'is_charging_station' 'status' 'ttl'] not found in axis"

Each month:
- Organize each row adding the date and hour of the update
- For each station:
    - For each hour (6.00 - 8.55) compute the mean along all days 
OUTPUT: a dataframe named monthly mean where each station should appear 24 times (one per hour)

In [8]:
# Compute the mean for each station
monthly_mean = pd.DataFrame(columns = ["station_id", "num_bikes_available", 
                                       "num_bikes_available_types.mechanical", 
                                       "num_bikes_available_types.ebike", "num_docks_available", "hour"])
not_used_stations = []

for station_id in data.station_id.unique():
    station_df = data[data["station_id"] == station_id]
    if len(station_df) >= 288 * 20: # minimum number of rows per station_id 
        for i in range(0, 24, 3):
                hours = ["{:02d}".format(i), "{:02d}".format(i+1), "{:02d}".format(i+2)]
                temp_df = station_df[station_df["hour"].str[:2].isin(hours)]
                mean = temp_df.mean().drop(labels=["day"])
                mean["hour"] = hours[0] + ":00 - " + hours[-1]+":55"
                mean["station_id"] = int(mean["station_id"])
                monthly_mean = monthly_mean.append(mean, ignore_index = True)
    else: not_used_stations.append(station_id)

In [9]:
monthly_mean

,station_id,num_bikes_available,num_bikes_available_types.mechanical,num_bikes_available_types.ebike,num_docks_available,hour
0,1,19.001854,18.802595,0.199259,23.582947,00:00 - 02:55
1,1,20.058333,19.862037,0.196296,22.656481,03:00 - 05:55
2,1,16.810185,17.160185,0.167593,25.395370,06:00 - 08:55
3,1,7.368519,7.492593,0.142593,35.060185,09:00 - 11:55
4,1,6.756030,6.614100,0.298701,35.893321,12:00 - 14:55
...,...,...,...,...,...,...
3923,330,0.427056,0.057029,0.458886,25.820955,09:00 - 11:55
3924,330,0.417772,0.030504,0.387268,25.885942,12:00 - 14:55
3925,330,0.288360,0.006614,0.281746,25.984127,15:00 - 17:55
3926,330,0.304233,0.017196,0.287037,25.818783,18:00 - 20:55


In [10]:
not_used_stations

[122, 430, 529, 500, 456, 449, 399]

In [12]:
len(monthly_mean.station_id.unique())

491

In [48]:
monthly_mean.hour.value_counts()

00:00 - 02:55    491
09:00 - 11:55    491
12:00 - 14:55    491
03:00 - 05:55    491
15:00 - 17:55    491
18:00 - 20:55    491
06:00 - 08:55    491
21:00 - 23:55    491
Name: hour, dtype: int64

In [61]:
test = data[data["station_id"] == 1]
mean_test = test[test["hour"].str[:2] == "08"].mean().drop(labels=["day"])
mean_test["hour"] = "08"
print(int(mean_test["station_id"]))

monthly_mean = monthly_mean.append(mean_test, ignore_index=True)
monthly_mean

1


,station_id,num_bikes_available,num_bikes_available_types.mechanical,num_bikes_available_types.ebike,num_docks_available,hour
0,1.0,17.641667,17.297222,0.344444,24.919444,00
1,1.0,19.316667,19.172222,0.144444,23.227778,01
2,1.0,20.050139,19.941504,0.108635,22.598886,02
3,1.0,20.233333,20.022222,0.211111,22.447222,03
4,1.0,20.055556,19.852778,0.202778,22.647222,04
...,...,...,...,...,...,...
11951,399.0,3.958333,2.958333,1.000000,23.000000,23
11952,1.0,14.308333,14.763889,0.183333,27.708333,08
11953,1.0,14.308333,14.763889,0.183333,27.708333,08
11954,1.0,14.308333,14.763889,0.183333,27.708333,08


## PROVES

In [31]:
# Compute the mean for each station
monthly_mean = pd.DataFrame(columns = ["station_id", "num_bikes_available", 
                                       "num_bikes_available_types.mechanical", 
                                       "num_bikes_available_types.ebike", "num_docks_available", "hour"])

station_df = data[data["station_id"] == 1]
for i in range(0, 24, 3):
    hours = ["{:02d}".format(i), "{:02d}".format(i+1), "{:02d}".format(i+2)]
    temp_df = station_df[station_df["hour"].str[:2].isin(hours)]
    mean = temp_df.mean().drop(labels=["day"])
    mean["hour"] = hours[0] + ":00 - " + hours[-1]+":55"
    mean["station_id"] = int(mean["station_id"])
    monthly_mean = monthly_mean.append(mean, ignore_index = True)

0
3
6
9
12
15
18
21


Get common stations --> those that appear in all datasets within one year. 